In [1]:
!pip install pykan

In [2]:
!pip install optuna

In [3]:
import optuna

In [4]:
import numpy as np
from scipy.stats import ks_2samp
import torch
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_curve, precision_recall_curve
)
import matplotlib.pyplot as plt

In [5]:
from kan import KAN

In [6]:
from pre_processing import  ChurnDataProcessor

In [7]:
processor = ChurnDataProcessor("customer_churn_telecom_services.csv")

In [8]:
X_train, y_train, X_val, y_val, X_test, y_test = processor.run_pipeline()


Iniciando Pipeline de Dados...
Divisão e Balanceamento concluídos.
--- Análise de Outliers (IQR) no Treino ---
Col TotalCharges: 24 outliers encontrados.
Removendo total de 24 registros...


In [ ]:
def train_model(
    model,
    X_train, y_train,
    X_val, y_val,
    epochs,
    lr,
    patience=20,
    min_delta=1e-4
):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCELoss()

    # Conversão segura
    if hasattr(X_train, "values"):
        X_train = X_train.values
    if hasattr(y_train, "values"):
        y_train = y_train.values
    if hasattr(X_val, "values"):
        X_val = X_val.values
    if hasattr(y_val, "values"):
        y_val = y_val.values

    Xtr = torch.tensor(X_train, dtype=torch.float32)
    ytr = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

    best_ks = -np.inf
    patience_counter = 0

    history = {
        "loss": [],
        "ks": []
    }

    for epoch in range(epochs):
        # ---- TREINO ----
        model.train()
        optimizer.zero_grad()

        logits = model(Xtr)
        preds = torch.sigmoid(logits)
        loss = loss_fn(preds, ytr)

        loss.backward()
        optimizer.step()

        # ---- VALIDAÇÃO (KS) ----
        model.eval()
        with torch.no_grad():
            val_logits = model(torch.tensor(X_val, dtype=torch.float32))
            val_preds = torch.sigmoid(val_logits).numpy().flatten()

        ks = ks_2samp(
            val_preds[y_val == 1],
            val_preds[y_val == 0]
        ).statistic

        history["loss"].append(loss.item())
        history["ks"].append(ks)

        # ---- LOG ----
        if epoch % 10 == 0 or epoch == epochs - 1:
            print(
                f"Epoch {epoch}/{epochs} | "
                f"Loss: {loss.item():.4f} | KS_val: {ks:.4f}"
            )

        # ---- EARLY STOPPING ----
        if ks > best_ks + min_delta:
            best_ks = ks
            patience_counter = 0
            best_state = model.state_dict()
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(
                f"Early stopping at epoch {epoch} | "
                f"Best KS: {best_ks:.4f}"
            )
            break

    model.load_state_dict(best_state)
    return history


In [ ]:
#@title Plot dos Gráficos
def plot_confusion_matrix(cm, title="Confusion Matrix"):
    plt.figure()
    plt.imshow(cm)
    plt.title(title)
    plt.colorbar()
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha="center", va="center")

    plt.show()
def plot_roc_curve(y_true, preds):
    fpr, tpr, _ = roc_curve(y_true, preds)
    auc_score = roc_auc_score(y_true, preds)

    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc_score:.3f}")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()
def plot_ks_curve(y_true, preds):
    scores_pos = preds[y_true == 1]
    scores_neg = preds[y_true == 0]

    bins = np.linspace(0, 1, 100)

    cdf_pos = np.cumsum(np.histogram(scores_pos, bins=bins)[0]) / len(scores_pos)
    cdf_neg = np.cumsum(np.histogram(scores_neg, bins=bins)[0]) / len(scores_neg)

    ks_curve = np.abs(cdf_pos - cdf_neg)

    plt.figure()
    plt.plot(bins[1:], cdf_pos, label="Positive class")
    plt.plot(bins[1:], cdf_neg, label="Negative class")
    plt.plot(bins[1:], ks_curve, label="KS distance", linestyle="--")
    plt.xlabel("Score")
    plt.ylabel("CDF")
    plt.title("KS Curve")
    plt.legend()
    plt.show()
def plot_ks_epochs(history):
    plt.figure()
    plt.plot(history["ks"])
    plt.xlabel("Epoch")
    plt.ylabel("KS (Validation)")
    plt.title("KS vs Epochs (Early Stopping)")
    plt.show()


In [ ]:
#@title Avaliação

In [ ]:
def evaluate_model(model, X, y, set_name="Validation"):
    model.eval()

    if hasattr(X, "values"):
        X = X.values
    if hasattr(y, "values"):
        y = y.values

    with torch.no_grad():
        logits = model(torch.tensor(X, dtype=torch.float32))
        preds = torch.sigmoid(logits).numpy().flatten()

    preds_binary = (preds >= 0.5).astype(int)

    ks_score = ks_2samp(preds[y == 1], preds[y == 0]).statistic
    auc = roc_auc_score(y, preds)
    precision = precision_score(y, preds_binary)
    recall = recall_score(y, preds_binary)
    f1 = f1_score(y, preds_binary)
    cm = confusion_matrix(y, preds_binary)

    print(f"\n---- {set_name} ----")
    print("KS:", ks_score)
    print("AUC:", auc)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1)
    print("Confusion Matrix:\n", cm)

    return {
        "ks": ks_score,
        "auc": auc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": cm,
        "preds": preds
    }


In [ ]:
#@title Baseline

In [ ]:
def objective_baseline(trial):
    h = trial.suggest_int("h", 8, 16)

    model = KAN(
        width=[X_train.shape[1], h, 1],
        grid=trial.suggest_int("grid", 4, 6),
        k=trial.suggest_int("k", 2, 4)
    )

    epochs = trial.suggest_int("epochs", 200, 500)
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)

    history = train_model(
        model,
        X_train, y_train,
        X_val, y_val,
        epochs=epochs,
        lr=lr,
        patience=20
    )

    return evaluate_model(model, X_val, y_val)["ks"]


In [ ]:
study_baseline = optuna.create_study(direction="maximize")
study_baseline.optimize(objective_baseline, n_trials=20)


[I 2025-12-13 16:19:34,662] A new study created in memory with name: no-name-ff09ecdb-b90a-44e2-baf7-56db3ccaedb4


checkpoint directory created: ./model
saving model version 0.0
Epoch 0/440 | Loss: 0.6906 | KS_val: 0.2158
Epoch 10/440 | Loss: 0.6783 | KS_val: 0.3241
Epoch 20/440 | Loss: 0.6658 | KS_val: 0.4161
Epoch 30/440 | Loss: 0.6525 | KS_val: 0.4609
Epoch 40/440 | Loss: 0.6381 | KS_val: 0.4695
Epoch 50/440 | Loss: 0.6226 | KS_val: 0.4756
Epoch 60/440 | Loss: 0.6063 | KS_val: 0.4803
Epoch 70/440 | Loss: 0.5898 | KS_val: 0.4811
Epoch 80/440 | Loss: 0.5737 | KS_val: 0.4849
Epoch 90/440 | Loss: 0.5592 | KS_val: 0.4872
Epoch 100/440 | Loss: 0.5470 | KS_val: 0.4919
Epoch 110/440 | Loss: 0.5370 | KS_val: 0.4919
Epoch 120/440 | Loss: 0.5291 | KS_val: 0.4942
Epoch 130/440 | Loss: 0.5229 | KS_val: 0.4934
Epoch 140/440 | Loss: 0.5180 | KS_val: 0.4973
Epoch 150/440 | Loss: 0.5138 | KS_val: 0.4988
Epoch 160/440 | Loss: 0.5103 | KS_val: 0.5043
Epoch 170/440 | Loss: 0.5072 | KS_val: 0.5081
Epoch 180/440 | Loss: 0.5045 | KS_val: 0.5104
Epoch 190/440 | Loss: 0.5021 | KS_val: 0.5120
Epoch 200/440 | Loss: 0.5000

[I 2025-12-13 16:23:11,111] Trial 0 finished with value: 0.5119876256767208 and parameters: {'h': 13, 'grid': 6, 'k': 3, 'epochs': 440, 'lr': 0.000564696875014404}. Best is trial 0 with value: 0.5119876256767208.



---- Validation ----
KS: 0.5119876256767208
AUC: 0.827985661384491
Precision: 0.7364228819695873
Recall: 0.7865429234338747
F1: 0.7606581899775617
Confusion Matrix:
 [[ 929  364]
 [ 276 1017]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/441 | Loss: 0.6947 | KS_val: 0.1075
Epoch 10/441 | Loss: 0.6921 | KS_val: 0.0874
Epoch 20/441 | Loss: 0.6896 | KS_val: 0.1701
Epoch 30/441 | Loss: 0.6871 | KS_val: 0.2367
Epoch 40/441 | Loss: 0.6847 | KS_val: 0.2776
Epoch 50/441 | Loss: 0.6821 | KS_val: 0.3086
Epoch 60/441 | Loss: 0.6795 | KS_val: 0.3326
Epoch 70/441 | Loss: 0.6768 | KS_val: 0.3581
Epoch 80/441 | Loss: 0.6740 | KS_val: 0.3836
Epoch 90/441 | Loss: 0.6711 | KS_val: 0.4029
Epoch 100/441 | Loss: 0.6681 | KS_val: 0.4114
Epoch 110/441 | Loss: 0.6649 | KS_val: 0.4285
Epoch 120/441 | Loss: 0.6616 | KS_val: 0.4316
Epoch 130/441 | Loss: 0.6582 | KS_val: 0.4346
Epoch 140/441 | Loss: 0.6547 | KS_val: 0.4416
Epoch 150/441 | Loss: 0.6511 | KS_val: 0.4447
Epoch 160/441 | Lo

[I 2025-12-13 16:28:24,762] Trial 1 finished with value: 0.46867749419953597 and parameters: {'h': 16, 'grid': 4, 'k': 4, 'epochs': 441, 'lr': 0.00011303340127159383}. Best is trial 0 with value: 0.5119876256767208.


...


---- Validation ----
KS: 0.4918793503480278
AUC: 0.803562403063913
Precision: 0.7011795543905636
Recall: 0.8275328692962104
F1: 0.7591344448385953
Confusion Matrix:
 [[ 837  456]
 [ 223 1070]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/396 | Loss: 0.6990 | KS_val: 0.3589
Epoch 10/396 | Loss: 0.6965 | KS_val: 0.2568
Epoch 20/396 | Loss: 0.6942 | KS_val: 0.0797
Early stopping at epoch 20 | Best KS: 0.3589


[I 2025-12-13 16:53:07,654] Trial 18 finished with value: 0.07965970610982212 and parameters: {'h': 10, 'grid': 4, 'k': 2, 'epochs': 396, 'lr': 0.00020445385039566106}. Best is trial 4 with value: 0.5150812064965197.



---- Validation ----
KS: 0.07965970610982212
AUC: 0.4617384704001378
Precision: 0.48886532343584305
Recall: 0.7130703789636504
F1: 0.580056621579113
Confusion Matrix:
 [[329 964]
 [371 922]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/317 | Loss: 0.6987 | KS_val: 0.1531
Epoch 10/317 | Loss: 0.6855 | KS_val: 0.2691
Epoch 20/317 | Loss: 0.6722 | KS_val: 0.3991
Epoch 30/317 | Loss: 0.6579 | KS_val: 0.4501
Epoch 40/317 | Loss: 0.6420 | KS_val: 0.4633
Epoch 50/317 | Loss: 0.6245 | KS_val: 0.4741
Epoch 60/317 | Loss: 0.6058 | KS_val: 0.4764
Epoch 70/317 | Loss: 0.5866 | KS_val: 0.4772
Early stopping at epoch 73 | Best KS: 0.4780


[I 2025-12-13 16:54:32,714] Trial 19 finished with value: 0.4779582366589327 and parameters: {'h': 16, 'grid': 6, 'k': 3, 'epochs': 317, 'lr': 0.0006208340692661691}. Best is trial 4 with value: 0.5150812064965197.



---- Validation ----
KS: 0.4779582366589327
AUC: 0.8035911137907789
Precision: 0.6951768488745981
Recall: 0.8360402165506574
F1: 0.7591292134831461
Confusion Matrix:
 [[ 819  474]
 [ 212 1081]]


In [ ]:
best_params = study_baseline.best_params
best_value = study_baseline.best_value

print("Melhores parâmetros:", best_params)
print("Melhor KS:", best_value)


Melhores parâmetros: {'h': 8, 'grid': 4, 'k': 3, 'epochs': 331, 'lr': 0.0005059511082207461}
Melhor KS: 0.5150812064965197


In [ ]:
#@title Experimento 1 -  Experimento de Topologia

In [ ]:
def objective_exp1(trial):
    # Capacidade da topologia
    h = trial.suggest_int("hidden_units", 4, 32)

    model = KAN(
        width=[X_train.shape[1], h, 1],
        grid=trial.suggest_int("grid", 4, 6),
        k=trial.suggest_int("k", 2, 4)
    )

    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)

    history = train_model(
        model,
        X_train, y_train,
        X_val, y_val,
        epochs=500,
        lr=lr,
        patience=20
    )

    metrics = evaluate_model(model, X_val, y_val, "Validation")
    return metrics["ks"]



In [ ]:
study_exp1 = optuna.create_study(direction="maximize")
study_exp1.optimize(objective_exp1, n_trials=20)

print("\nMelhor configuração Experimento 1:")
print(study_exp1.best_params)
print("Best KS:", study_exp1.best_value)

[I 2025-12-13 17:06:51,191] A new study created in memory with name: no-name-bbdd4629-5024-401a-914a-1c656fa9a37c


checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6928 | KS_val: 0.0518
Epoch 10/500 | Loss: 0.6881 | KS_val: 0.2722
Epoch 20/500 | Loss: 0.6834 | KS_val: 0.3968
Epoch 30/500 | Loss: 0.6786 | KS_val: 0.4486
Epoch 40/500 | Loss: 0.6736 | KS_val: 0.4695
Epoch 50/500 | Loss: 0.6684 | KS_val: 0.4725
Epoch 60/500 | Loss: 0.6630 | KS_val: 0.4749
Epoch 70/500 | Loss: 0.6571 | KS_val: 0.4841
Epoch 80/500 | Loss: 0.6510 | KS_val: 0.4965
Epoch 90/500 | Loss: 0.6445 | KS_val: 0.4911
Epoch 100/500 | Loss: 0.6377 | KS_val: 0.4927
Early stopping at epoch 101 | Best KS: 0.4973


[I 2025-12-13 17:08:28,963] Trial 0 finished with value: 0.4934261407579273 and parameters: {'hidden_units': 13, 'grid': 4, 'k': 2, 'lr': 0.00022637828147556308}. Best is trial 0 with value: 0.4934261407579273.



---- Validation ----
KS: 0.4934261407579273
AUC: 0.8075017540459694
Precision: 0.693745970341715
Recall: 0.8321732405259087
F1: 0.7566807313642757
Confusion Matrix:
 [[ 818  475]
 [ 217 1076]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6858 | KS_val: 0.3689
Epoch 10/500 | Loss: 0.6755 | KS_val: 0.4633
Epoch 20/500 | Loss: 0.6649 | KS_val: 0.4841
Epoch 30/500 | Loss: 0.6537 | KS_val: 0.4927
Epoch 40/500 | Loss: 0.6414 | KS_val: 0.4942
Epoch 50/500 | Loss: 0.6282 | KS_val: 0.4934
Early stopping at epoch 56 | Best KS: 0.4957


[I 2025-12-13 17:10:11,201] Trial 1 finished with value: 0.494199535962877 and parameters: {'hidden_units': 23, 'grid': 5, 'k': 3, 'lr': 0.0003995462774128058}. Best is trial 1 with value: 0.494199535962877.


...


---- Validation ----
KS: 0.505800464037123
AUC: 0.8231335485441569
Precision: 0.7317784256559767
Recall: 0.7764887857695282
F1: 0.7534709193245779
Confusion Matrix:
 [[ 925  368]
 [ 289 1004]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6861 | KS_val: 0.2498
Epoch 10/500 | Loss: 0.6788 | KS_val: 0.3241
Epoch 20/500 | Loss: 0.6716 | KS_val: 0.3859
Epoch 30/500 | Loss: 0.6644 | KS_val: 0.4339
Epoch 40/500 | Loss: 0.6571 | KS_val: 0.4555
Epoch 50/500 | Loss: 0.6496 | KS_val: 0.4780
Epoch 60/500 | Loss: 0.6417 | KS_val: 0.4872
Epoch 70/500 | Loss: 0.6336 | KS_val: 0.4903
Epoch 80/500 | Loss: 0.6252 | KS_val: 0.4919
Epoch 90/500 | Loss: 0.6165 | KS_val: 0.4927
Early stopping at epoch 96 | Best KS: 0.4950


[I 2025-12-13 18:10:52,593] Trial 18 finished with value: 0.494199535962877 and parameters: {'hidden_units': 18, 'grid': 5, 'k': 2, 'lr': 0.0002603922726767856}. Best is trial 16 with value: 0.5243619489559165.



---- Validation ----
KS: 0.494199535962877
AUC: 0.8021328481220494
Precision: 0.6700782661047562
Recall: 0.8607888631090487
F1: 0.7535545023696683
Confusion Matrix:
 [[ 745  548]
 [ 180 1113]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.7077 | KS_val: 0.3016
Epoch 10/500 | Loss: 0.6900 | KS_val: 0.2305
Epoch 20/500 | Loss: 0.6739 | KS_val: 0.4439
Epoch 30/500 | Loss: 0.6582 | KS_val: 0.4493
Epoch 40/500 | Loss: 0.6421 | KS_val: 0.4687
Epoch 50/500 | Loss: 0.6251 | KS_val: 0.4702
Epoch 60/500 | Loss: 0.6075 | KS_val: 0.4795
Epoch 70/500 | Loss: 0.5897 | KS_val: 0.4772
Epoch 80/500 | Loss: 0.5727 | KS_val: 0.4803
Epoch 90/500 | Loss: 0.5573 | KS_val: 0.4865
Epoch 100/500 | Loss: 0.5443 | KS_val: 0.4888
Epoch 110/500 | Loss: 0.5339 | KS_val: 0.4896
Epoch 120/500 | Loss: 0.5259 | KS_val: 0.4942
Epoch 130/500 | Loss: 0.5198 | KS_val: 0.4965
Epoch 140/500 | Loss: 0.5152 | KS_val: 0.4973
Epoch 150/500 | Loss: 0.5115 | KS_val: 0.4965
Early stopping at 

[I 2025-12-13 18:17:10,455] Trial 19 finished with value: 0.4980665119876257 and parameters: {'hidden_units': 32, 'grid': 5, 'k': 3, 'lr': 0.0004558758817984067}. Best is trial 16 with value: 0.5243619489559165.



---- Validation ----
KS: 0.4980665119876257
AUC: 0.820727828888853
Precision: 0.7210119465917076
Recall: 0.7935034802784223
F1: 0.7555228276877761
Confusion Matrix:
 [[ 896  397]
 [ 267 1026]]

Melhor configuração Experimento 1:
{'hidden_units': 28, 'grid': 5, 'k': 2, 'lr': 0.0007081228143037425}
Best KS: 0.5243619489559165


In [ ]:
#@title Experimento 2 - Parâmetros Estruturais

In [ ]:
# Melhor largura encontrada no Experimento 1
BEST_H = study_exp1.best_params["hidden_units"]


In [ ]:
def objective_exp2(trial):
    model = KAN(
        width=[X_train.shape[1], BEST_H, 1],
        grid=trial.suggest_int("grid", 3, 8),
        k=trial.suggest_int("k", 2, 6)
    )

    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)

    train_model(
        model,
        X_train, y_train,
        X_val, y_val,
        epochs=500,
        lr=lr,
        patience=20
    )

    metrics = evaluate_model(model, X_val, y_val, "Validation")
    return metrics["ks"]


In [ ]:
study_exp2 = optuna.create_study(direction="maximize")
study_exp2.optimize(objective_exp2, n_trials=25)

print("\n===== EXPERIMENTO 2 =====")
print("Melhores hiperparâmetros:", study_exp2.best_params)
print("Melhor KS:", study_exp2.best_value)


[I 2025-12-13 18:39:50,760] A new study created in memory with name: no-name-03c1e796-8418-4647-a69e-3e3735450991


checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6988 | KS_val: 0.1802
Epoch 10/500 | Loss: 0.6843 | KS_val: 0.3542
Epoch 20/500 | Loss: 0.6710 | KS_val: 0.4238
Epoch 30/500 | Loss: 0.6574 | KS_val: 0.4416
Epoch 40/500 | Loss: 0.6425 | KS_val: 0.4563
Epoch 50/500 | Loss: 0.6261 | KS_val: 0.4602
Epoch 60/500 | Loss: 0.6088 | KS_val: 0.4733
Epoch 70/500 | Loss: 0.5914 | KS_val: 0.4780
Epoch 80/500 | Loss: 0.5749 | KS_val: 0.4795
Epoch 90/500 | Loss: 0.5603 | KS_val: 0.4818
Epoch 100/500 | Loss: 0.5480 | KS_val: 0.4865
Epoch 110/500 | Loss: 0.5381 | KS_val: 0.4857
Epoch 120/500 | Loss: 0.5303 | KS_val: 0.4865
Epoch 130/500 | Loss: 0.5242 | KS_val: 0.4896
Epoch 140/500 | Loss: 0.5193 | KS_val: 0.4903
Epoch 150/500 | Loss: 0.5153 | KS_val: 0.4942
Epoch 160/500 | Loss: 0.5120 | KS_val: 0.4981
Epoch 170/500 | Loss: 0.5091 | KS_val: 0.4996
Epoch 180/500 | Loss: 0.5067 | KS_val: 0.5019
Epoch 190/500 | Loss: 0.5046 | KS_val: 0.5019
Epoch 200/500 | Loss: 0.5028

[I 2025-12-13 18:49:10,874] Trial 0 finished with value: 0.5135344160866203 and parameters: {'grid': 4, 'k': 4, 'lr': 0.0004569088482678212}. Best is trial 0 with value: 0.5135344160866203.



---- Validation ----
KS: 0.5135344160866203
AUC: 0.8277834900161438
Precision: 0.7343862167982771
Recall: 0.7911832946635731
F1: 0.7617274758004468
Confusion Matrix:
 [[ 923  370]
 [ 270 1023]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6993 | KS_val: 0.2243
Epoch 10/500 | Loss: 0.6935 | KS_val: 0.0967
Epoch 20/500 | Loss: 0.6881 | KS_val: 0.2900
Epoch 30/500 | Loss: 0.6829 | KS_val: 0.3782
Epoch 40/500 | Loss: 0.6778 | KS_val: 0.4153
Epoch 50/500 | Loss: 0.6725 | KS_val: 0.4192
Epoch 60/500 | Loss: 0.6671 | KS_val: 0.4308
Epoch 70/500 | Loss: 0.6614 | KS_val: 0.4370
Epoch 80/500 | Loss: 0.6554 | KS_val: 0.4424
Epoch 90/500 | Loss: 0.6490 | KS_val: 0.4478
Epoch 100/500 | Loss: 0.6422 | KS_val: 0.4501
Epoch 110/500 | Loss: 0.6351 | KS_val: 0.4524
Epoch 120/500 | Loss: 0.6277 | KS_val: 0.4617
Epoch 130/500 | Loss: 0.6200 | KS_val: 0.4656
Epoch 140/500 | Loss: 0.6120 | KS_val: 0.4671
Epoch 150/500 | Loss: 0.6041 | KS_val: 0.4718
Epoch 160/500 | L

[I 2025-12-13 18:59:24,069] Trial 1 finished with value: 0.48878576952822894 and parameters: {'grid': 4, 'k': 5, 'lr': 0.00019054613966560754}. Best is trial 0 with value: 0.5135344160866203.


...


---- Validation ----
KS: 0.5150812064965197
AUC: 0.8317706922096432
Precision: 0.7371469949312093
Recall: 0.7873163186388245
F1: 0.7614061331338818
Confusion Matrix:
 [[ 930  363]
 [ 275 1018]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6986 | KS_val: 0.1446
Epoch 10/500 | Loss: 0.6751 | KS_val: 0.4169
Epoch 20/500 | Loss: 0.6531 | KS_val: 0.4517
Epoch 30/500 | Loss: 0.6285 | KS_val: 0.4625
Epoch 40/500 | Loss: 0.6011 | KS_val: 0.4718
Epoch 50/500 | Loss: 0.5740 | KS_val: 0.4772
Epoch 60/500 | Loss: 0.5513 | KS_val: 0.4811
Epoch 70/500 | Loss: 0.5349 | KS_val: 0.4849
Epoch 80/500 | Loss: 0.5241 | KS_val: 0.4872
Epoch 90/500 | Loss: 0.5167 | KS_val: 0.4934
Epoch 100/500 | Loss: 0.5112 | KS_val: 0.5004
Epoch 110/500 | Loss: 0.5068 | KS_val: 0.5012
Epoch 120/500 | Loss: 0.5034 | KS_val: 0.5050
Epoch 130/500 | Loss: 0.5006 | KS_val: 0.5089
Epoch 140/500 | Loss: 0.4982 | KS_val: 0.5143
Epoch 150/500 | Loss: 0.4961 | KS_val: 0.5151
Epoch 160/500 | L

[I 2025-12-13 20:56:04,885] Trial 23 finished with value: 0.5150812064965197 and parameters: {'grid': 4, 'k': 3, 'lr': 0.0007760363945313869}. Best is trial 21 with value: 0.5150812064965197.



---- Validation ----
KS: 0.5150812064965197
AUC: 0.8300019918066763
Precision: 0.7341587764020393
Recall: 0.7795823665893271
F1: 0.7561890472618155
Confusion Matrix:
 [[ 928  365]
 [ 285 1008]]
checkpoint directory created: ./model
saving model version 0.0
Epoch 0/500 | Loss: 0.6991 | KS_val: 0.1640
Epoch 10/500 | Loss: 0.6751 | KS_val: 0.4161
Epoch 20/500 | Loss: 0.6523 | KS_val: 0.4524
Epoch 30/500 | Loss: 0.6268 | KS_val: 0.4609
Epoch 40/500 | Loss: 0.5991 | KS_val: 0.4741
Epoch 50/500 | Loss: 0.5726 | KS_val: 0.4834
Epoch 60/500 | Loss: 0.5508 | KS_val: 0.4865
Epoch 70/500 | Loss: 0.5350 | KS_val: 0.4849
Epoch 80/500 | Loss: 0.5244 | KS_val: 0.4911
Epoch 90/500 | Loss: 0.5169 | KS_val: 0.4903
Epoch 100/500 | Loss: 0.5112 | KS_val: 0.4973
Epoch 110/500 | Loss: 0.5068 | KS_val: 0.5012
Epoch 120/500 | Loss: 0.5033 | KS_val: 0.5058
Epoch 130/500 | Loss: 0.5004 | KS_val: 0.5081
Epoch 140/500 | Loss: 0.4981 | KS_val: 0.5104
Epoch 150/500 | Loss: 0.4960 | KS_val: 0.5128
Epoch 160/500 | L

[I 2025-12-13 21:02:18,326] Trial 24 finished with value: 0.5127610208816705 and parameters: {'grid': 4, 'k': 2, 'lr': 0.0007613437574679622}. Best is trial 21 with value: 0.5150812064965197.



---- Validation ----
KS: 0.5127610208816705
AUC: 0.8304488024935267
Precision: 0.7339581831290555
Recall: 0.7873163186388245
F1: 0.7597014925373134
Confusion Matrix:
 [[ 924  369]
 [ 275 1018]]

===== EXPERIMENTO 2 =====
Melhores hiperparâmetros: {'grid': 5, 'k': 3, 'lr': 0.0008034011145827787}
Melhor KS: 0.5150812064965197


In [ ]:
#@title Experimento 3 - Profundidade Estrutural

In [ ]:
def objective_exp3(trial):

    h1 = trial.suggest_int("h1", 16, 32)
    h2 = trial.suggest_int("h2", 8, 16)
    grid = trial.suggest_int("grid", 4, 6)
    k = 2
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)

    max_epochs = 500
    patience = 30
    min_delta = 1e-4

    # 🔹 conversão correta
    Xtr = torch.tensor(X_train.values, dtype=torch.float32)
    ytr = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

    Xval = torch.tensor(X_val.values, dtype=torch.float32)
    yval = y_val  # numpy (usado só no KS)

    model = KAN(
        width=[Xtr.shape[1], h1, h2, 1],
        grid=grid,
        k=k,
        seed=42
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    best_ks = -1.0
    patience_counter = 0

    print(
        f"\n[EXP3] Trial {trial.number} | "
        f"h1={h1}, h2={h2}, grid={grid}, lr={lr:.2e}"
    )

    for epoch in range(max_epochs):
        model.train()

        optimizer.zero_grad()
        logits = model(Xtr)
        loss = loss_fn(logits, ytr)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_logits = model(Xval)
            val_probs = torch.sigmoid(val_logits).cpu().numpy().flatten()

        ks = ks_2samp(
            val_probs[yval == 1],
            val_probs[yval == 0]
        ).statistic

        if epoch % 10 == 0:
            print(
                f"Epoch {epoch}/{max_epochs} | "
                f"Loss: {loss.item():.4f} | "
                f"KS_val: {ks:.4f}"
            )

        if ks > best_ks + min_delta:
            best_ks = ks
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(
                f"Early stopping at epoch {epoch} | "
                f"Best KS: {best_ks:.4f}"
            )
            break

    return best_ks


In [ ]:
study_exp3 = optuna.create_study(direction="maximize")
study_exp3.optimize(objective_exp3, n_trials=20)

print("Melhor KS EXP3:", study_exp3.best_value)
print("Melhores parâmetros EXP3:", study_exp3.best_params)


[I 2025-12-13 21:28:03,218] A new study created in memory with name: no-name-f5cd2e9a-eedc-42f7-84b5-3746cac8828c


checkpoint directory created: ./model
saving model version 0.0

[EXP3] Trial 0 | h1=18, h2=16, grid=4, lr=7.03e-04
Epoch 0/500 | Loss: 0.6916 | KS_val: 0.2854
Epoch 10/500 | Loss: 0.6810 | KS_val: 0.4772
Epoch 20/500 | Loss: 0.6666 | KS_val: 0.4841
Epoch 30/500 | Loss: 0.6451 | KS_val: 0.4911
Epoch 40/500 | Loss: 0.6168 | KS_val: 0.4934
Epoch 50/500 | Loss: 0.5851 | KS_val: 0.4981
Epoch 60/500 | Loss: 0.5559 | KS_val: 0.4981
Epoch 70/500 | Loss: 0.5340 | KS_val: 0.4957
Epoch 80/500 | Loss: 0.5206 | KS_val: 0.5019
Epoch 90/500 | Loss: 0.5130 | KS_val: 0.5089
Epoch 100/500 | Loss: 0.5074 | KS_val: 0.5151
Epoch 110/500 | Loss: 0.5028 | KS_val: 0.5143
Epoch 120/500 | Loss: 0.4993 | KS_val: 0.5135
Epoch 130/500 | Loss: 0.4965 | KS_val: 0.5135


[I 2025-12-13 21:32:07,661] Trial 0 finished with value: 0.5181747873163186 and parameters: {'h1': 18, 'h2': 16, 'grid': 4, 'lr': 0.0007025198936880276}. Best is trial 0 with value: 0.5181747873163186.


Early stopping at epoch 137 | Best KS: 0.5182
checkpoint directory created: ./model
saving model version 0.0

[EXP3] Trial 1 | h1=27, h2=14, grid=4, lr=2.94e-04
Epoch 0/500 | Loss: 0.6962 | KS_val: 0.1926
Epoch 10/500 | Loss: 0.6915 | KS_val: 0.1725
Epoch 20/500 | Loss: 0.6863 | KS_val: 0.3797
Epoch 30/500 | Loss: 0.6801 | KS_val: 0.4362
Epoch 40/500 | Loss: 0.6721 | KS_val: 0.4586
Epoch 50/500 | Loss: 0.6620 | KS_val: 0.4756
Epoch 60/500 | Loss: 0.6496 | KS_val: 0.4880
Epoch 70/500 | Loss: 0.6350 | KS_val: 0.4911
Epoch 80/500 | Loss: 0.6187 | KS_val: 0.4903
Epoch 90/500 | Loss: 0.6014 | KS_val: 0.4981
Epoch 100/500 | Loss: 0.5841 | KS_val: 0.4965
Epoch 110/500 | Loss: 0.5680 | KS_val: 0.4957


[I 2025-12-13 21:36:55,846] Trial 1 finished with value: 0.49961330239752516 and parameters: {'h1': 27, 'h2': 14, 'grid': 4, 'lr': 0.00029376142186453665}. Best is trial 0 with value: 0.5181747873163186.


...

Early stopping at epoch 146 | Best KS: 0.5244
checkpoint directory created: ./model
saving model version 0.0

[EXP3] Trial 18 | h1=30, h2=15, grid=5, lr=3.46e-04
Epoch 0/500 | Loss: 0.6936 | KS_val: 0.0526
Epoch 10/500 | Loss: 0.6877 | KS_val: 0.4501
Epoch 20/500 | Loss: 0.6812 | KS_val: 0.5027
Epoch 30/500 | Loss: 0.6728 | KS_val: 0.5004
Epoch 40/500 | Loss: 0.6618 | KS_val: 0.5012
Epoch 50/500 | Loss: 0.6477 | KS_val: 0.5012


[I 2025-12-13 23:31:05,347] Trial 18 finished with value: 0.5050270688321732 and parameters: {'h1': 30, 'h2': 15, 'grid': 5, 'lr': 0.00034592599342211255}. Best is trial 11 with value: 0.531322505800464.


Early stopping at epoch 51 | Best KS: 0.5050
checkpoint directory created: ./model
saving model version 0.0

[EXP3] Trial 19 | h1=32, h2=11, grid=6, lr=7.75e-04
Epoch 0/500 | Loss: 0.6915 | KS_val: 0.3279
Epoch 10/500 | Loss: 0.6820 | KS_val: 0.4517
Epoch 20/500 | Loss: 0.6676 | KS_val: 0.4811
Epoch 30/500 | Loss: 0.6442 | KS_val: 0.4803
Epoch 40/500 | Loss: 0.6113 | KS_val: 0.4841
Epoch 50/500 | Loss: 0.5762 | KS_val: 0.4818


[I 2025-12-13 23:33:32,155] Trial 19 finished with value: 0.48491879350348027 and parameters: {'h1': 32, 'h2': 11, 'grid': 6, 'lr': 0.0007753693750404529}. Best is trial 11 with value: 0.531322505800464.


Early stopping at epoch 51 | Best KS: 0.4849
Melhor KS EXP3: 0.531322505800464
Melhores parâmetros EXP3: {'h1': 32, 'h2': 11, 'grid': 6, 'lr': 0.00047592015963147515}


In [ ]:
#@title Experimento 4- Regularização

In [9]:
def objective_exp4(trial):

    h1 = trial.suggest_int("h1", 24, 36)
    h2 = trial.suggest_int("h2", 10, 18)
    grid = trial.suggest_int("grid", 4, 6)
    k = 2  # fixo para controle de complexidade

    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)
    weight_decay = trial.suggest_float(
        "weight_decay", 1e-6, 1e-3, log=True
    )


    max_epochs = 500
    patience = 30
    min_delta = 1e-4


    Xtr = torch.tensor(
        X_train if isinstance(X_train, np.ndarray) else X_train.values,
        dtype=torch.float32
    )
    ytr = torch.tensor(
        y_train if isinstance(y_train, np.ndarray) else y_train.values,
        dtype=torch.float32
    ).view(-1, 1)

    Xval = torch.tensor(
        X_val if isinstance(X_val, np.ndarray) else X_val.values,
        dtype=torch.float32
    )
    yval = y_val  # numpy (usado apenas no KS)


    model = KAN(
        width=[Xtr.shape[1], h1, h2, 1],
        grid=grid,
        k=k,
        seed=42
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )

    loss_fn = torch.nn.BCEWithLogitsLoss()

    best_ks = -1.0
    patience_counter = 0

    print(
        f"\n[EXP4] Trial {trial.number} | "
        f"h1={h1}, h2={h2}, grid={grid}, "
        f"lr={lr:.2e}, wd={weight_decay:.2e}"
    )


    for epoch in range(max_epochs):
        model.train()

        optimizer.zero_grad()
        logits = model(Xtr)
        loss = loss_fn(logits, ytr)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_logits = model(Xval)
            val_probs = torch.sigmoid(val_logits).cpu().numpy().flatten()

        ks = ks_2samp(
            val_probs[yval == 1],
            val_probs[yval == 0]
        ).statistic

        if epoch % 10 == 0:
            print(
                f"Epoch {epoch}/{max_epochs} | "
                f"Loss: {loss.item():.4f} | "
                f"KS_val: {ks:.4f}"
            )


        if ks > best_ks + min_delta:
            best_ks = ks
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(
                f"Early stopping at epoch {epoch} | "
                f"Best KS: {best_ks:.4f}"
            )
            break

    return best_ks


In [10]:
study_exp4 = optuna.create_study(direction="maximize")
study_exp4.optimize(objective_exp4, n_trials=30)

print("Melhor KS EXP4:", study_exp4.best_value)
print("Melhores parâmetros EXP4:", study_exp4.best_params)


[I 2025-12-14 03:10:20,669] A new study created in memory with name: no-name-702c83db-078d-478c-a929-7253810c9b43


checkpoint directory created: ./model
saving model version 0.0

[EXP4] Trial 0 | h1=36, h2=15, grid=6, lr=3.47e-04, wd=2.70e-04
Epoch 0/500 | Loss: 0.6939 | KS_val: 0.1353
Epoch 10/500 | Loss: 0.6872 | KS_val: 0.3210
Epoch 20/500 | Loss: 0.6797 | KS_val: 0.4076
Epoch 30/500 | Loss: 0.6702 | KS_val: 0.4455
Epoch 40/500 | Loss: 0.6580 | KS_val: 0.4578
Epoch 50/500 | Loss: 0.6426 | KS_val: 0.4617
Epoch 60/500 | Loss: 0.6246 | KS_val: 0.4640
Epoch 70/500 | Loss: 0.6055 | KS_val: 0.4725
Epoch 80/500 | Loss: 0.5866 | KS_val: 0.4756
Epoch 90/500 | Loss: 0.5690 | KS_val: 0.4811
Epoch 100/500 | Loss: 0.5534 | KS_val: 0.4857
Epoch 110/500 | Loss: 0.5403 | KS_val: 0.4950
Epoch 120/500 | Loss: 0.5299 | KS_val: 0.4988
Epoch 130/500 | Loss: 0.5217 | KS_val: 0.4996
Epoch 140/500 | Loss: 0.5153 | KS_val: 0.4988
Epoch 150/500 | Loss: 0.5101 | KS_val: 0.5019
Epoch 160/500 | Loss: 0.5057 | KS_val: 0.5043
Epoch 170/500 | Loss: 0.5021 | KS_val: 0.5120
Epoch 180/500 | Loss: 0.4991 | KS_val: 0.5143
Epoch 190

[I 2025-12-14 03:25:18,261] Trial 0 finished with value: 0.5212683681361175 and parameters: {'h1': 36, 'h2': 15, 'grid': 6, 'lr': 0.0003470307389831452, 'weight_decay': 0.00027023019149068214}. Best is trial 0 with value: 0.5212683681361175.


Early stopping at epoch 225 | Best KS: 0.5213
checkpoint directory created: ./model
saving model version 0.0

[EXP4] Trial 1 | h1=24, h2=15, grid=5, lr=4.60e-04, wd=1.67e-04
Epoch 0/500 | Loss: 0.6930 | KS_val: 0.0812
Epoch 10/500 | Loss: 0.6877 | KS_val: 0.3906
Epoch 20/500 | Loss: 0.6805 | KS_val: 0.4447
Epoch 30/500 | Loss: 0.6696 | KS_val: 0.4702
Epoch 40/500 | Loss: 0.6542 | KS_val: 0.4764
Epoch 50/500 | Loss: 0.6348 | KS_val: 0.4841
Epoch 60/500 | Loss: 0.6131 | KS_val: 0.4927
Epoch 70/500 | Loss: 0.5916 | KS_val: 0.5012
Epoch 80/500 | Loss: 0.5715 | KS_val: 0.4981
Epoch 90/500 | Loss: 0.5541 | KS_val: 0.4996


[I 2025-12-14 03:29:24,903] Trial 1 finished with value: 0.5011600928074246 and parameters: {'h1': 24, 'h2': 15, 'grid': 5, 'lr': 0.0004603336241443052, 'weight_decay': 0.00016700673829566013}. Best is trial 0 with value: 0.5212683681361175.


...

Early stopping at epoch 166 | Best KS: 0.5244
checkpoint directory created: ./model
saving model version 0.0

[EXP4] Trial 28 | h1=31, h2=14, grid=6, lr=8.43e-04, wd=1.82e-06
Epoch 0/500 | Loss: 0.6927 | KS_val: 0.2305
Epoch 10/500 | Loss: 0.6791 | KS_val: 0.4911
Epoch 20/500 | Loss: 0.6575 | KS_val: 0.4942
Epoch 30/500 | Loss: 0.6245 | KS_val: 0.4973
Epoch 40/500 | Loss: 0.5869 | KS_val: 0.4988
Epoch 50/500 | Loss: 0.5536 | KS_val: 0.4981
Epoch 60/500 | Loss: 0.5278 | KS_val: 0.5004
Epoch 70/500 | Loss: 0.5126 | KS_val: 0.5073
Epoch 80/500 | Loss: 0.5025 | KS_val: 0.5120
Epoch 90/500 | Loss: 0.4961 | KS_val: 0.5143
Epoch 100/500 | Loss: 0.4915 | KS_val: 0.5120
Epoch 110/500 | Loss: 0.4877 | KS_val: 0.5151
Epoch 120/500 | Loss: 0.4842 | KS_val: 0.5182
Epoch 130/500 | Loss: 0.4806 | KS_val: 0.5151
Epoch 140/500 | Loss: 0.4768 | KS_val: 0.5197
Epoch 150/500 | Loss: 0.4724 | KS_val: 0.5166
Epoch 160/500 | Loss: 0.4675 | KS_val: 0.5197
Epoch 170/500 | Loss: 0.4616 | KS_val: 0.5228
Epoch 18

[I 2025-12-14 07:10:34,182] Trial 28 finished with value: 0.5359628770301624 and parameters: {'h1': 31, 'h2': 14, 'grid': 6, 'lr': 0.0008426486867565753, 'weight_decay': 1.8217611922890154e-06}. Best is trial 11 with value: 0.5614849187935035.


Epoch 240/500 | Loss: 0.3774 | KS_val: 0.5104
Early stopping at epoch 240 | Best KS: 0.5360
checkpoint directory created: ./model
saving model version 0.0

[EXP4] Trial 29 | h1=35, h2=15, grid=6, lr=3.43e-04, wd=2.75e-06
Epoch 0/500 | Loss: 0.6973 | KS_val: 0.3766
Epoch 10/500 | Loss: 0.6906 | KS_val: 0.3372
Epoch 20/500 | Loss: 0.6838 | KS_val: 0.4478
Epoch 30/500 | Loss: 0.6751 | KS_val: 0.4695
Epoch 40/500 | Loss: 0.6631 | KS_val: 0.4818
Epoch 50/500 | Loss: 0.6472 | KS_val: 0.4834
Epoch 60/500 | Loss: 0.6278 | KS_val: 0.4834
Epoch 70/500 | Loss: 0.6065 | KS_val: 0.4872
Epoch 80/500 | Loss: 0.5854 | KS_val: 0.4903
Epoch 90/500 | Loss: 0.5663 | KS_val: 0.4919
Epoch 100/500 | Loss: 0.5502 | KS_val: 0.4965
Epoch 110/500 | Loss: 0.5376 | KS_val: 0.4973
Epoch 120/500 | Loss: 0.5280 | KS_val: 0.5035
Epoch 130/500 | Loss: 0.5208 | KS_val: 0.5043
Epoch 140/500 | Loss: 0.5151 | KS_val: 0.5012
Epoch 150/500 | Loss: 0.5104 | KS_val: 0.5081
Epoch 160/500 | Loss: 0.5064 | KS_val: 0.5097
Epoch 17

[I 2025-12-14 07:25:46,267] Trial 29 finished with value: 0.5189481825212684 and parameters: {'h1': 35, 'h2': 15, 'grid': 6, 'lr': 0.00034253525616349925, 'weight_decay': 2.7463164065698104e-06}. Best is trial 11 with value: 0.5614849187935035.


Early stopping at epoch 235 | Best KS: 0.5189
Melhor KS EXP4: 0.5614849187935035
Melhores parâmetros EXP4: {'h1': 28, 'h2': 13, 'grid': 6, 'lr': 0.0009451452764001935, 'weight_decay': 1.0097690060032607e-06}
